Install dependencies

In [1]:
!apt-get update
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
# install required system dependencies
!apt-get install -y xvfb x11-utils

# install required python dependencies (might need to install additional gym extras depending)
!pip install gym[box2d]==0.17.* pyvirtualdisplay==0.2.* PyOpenGL==3.1.* PyOpenGL-accelerate==3.1.*

!git clone https://github.com/openai/gym.git
%cd gym
!pip install -e .
!pip install stable-baselines[mpi]
!pip install stable-baselines3[extra]

%cd ..

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 https://developer.download.nvi

In [2]:
import os
import sys
import shutil

Running_in_colab = 'google.colab' in sys.modules

if Running_in_colab:
  sys_name = 'COLAB'
else:
  sys_name = os.environ['COMPUTERNAME']

# Importing Libs

!wget -q -O bestmodel.ckpt https://www.dropbox.com/s/qt0w4k2rqgo74xa/BestCarRacing_363_510.1083032490862_model.ckpt?dl=0
if not os.path.exists("lib"):
    print("LOADING FILES")

    if not os.path.exists("lib.zip"):
        !wget -q -O lib.zip https://www.dropbox.com/s/f1c8cl35uqahans/lib_2.zip?dl=0
    if not os.path.exists("lib"):
        !unzip -o "/content/lib.zip" -d "/content" 

LOADING FILES
Archive:  /content/lib.zip
   creating: /content/lib/
  inflating: /content/lib/archs.py   
 extracting: /content/lib/desktop.ini  
  inflating: /content/lib/my_tools.py  
  inflating: /content/lib/performance_evaluation.py  
  inflating: /content/lib/train_loop.py  
 extracting: /content/lib/__init__.py  


Imports

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import time
from matplotlib.pyplot import figure

import gym
import torch 
from torch.utils import data
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd
import random
from gym import wrappers
import copy
import gym
from gym.wrappers import Monitor
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay
Running_in_colab = 'google.colab' in sys.modules

display = Display(visible=0, size=(1400, 900))
display.start()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# from PartI_lib import archs as archs
from lib import performance_evaluation as eval
# from PartI_lib import train_loop as tl
from lib import my_tools as mt


###################################################
###################################################
###################################################
STUDENTID1=2016116214;
STUDENTID2=2016193815;
###################################################
###################################################
###################################################



Deep RL Replay Buffer

In [4]:
class ReplayBuffer:
    def __init__(self, size):
        self.size = size
        self.counter = 0
        self.state_buffer = []
        self.action_buffer = []
        self.reward_buffer = []
        self.new_state_buffer = []
        self.terminal_buffer = []
        
    def store_tuples(self, state, action, reward, new_state, done):
        if len(self.state_buffer) < self.size:
            self.state_buffer.append(None)
            self.action_buffer.append(None)
            self.reward_buffer.append(None)
            self.new_state_buffer.append(None)
            self.terminal_buffer.append(None)
            
        self.counter = self.counter % self.size
        self.state_buffer[self.counter] = state
        self.action_buffer[self.counter] = action
        self.reward_buffer[self.counter] = reward
        self.new_state_buffer[self.counter] = new_state
        self.terminal_buffer[self.counter] = done
        self.counter += 1

    def sample_buffer(self, batch_size):
        max_buffer = min(self.counter, self.size)
        batch = np.random.choice(max_buffer, batch_size, replace=False)
        state_batch=[];
        action_batch=[];
        reward_batch=[];
        new_state_batch=[];
        done_batch=[];
        
        for a in batch:
            state_batch.append(self.state_buffer[a])
            action_batch.append(self.action_buffer[a])
            reward_batch.append(self.reward_buffer[a])
            new_state_batch.append(self.new_state_buffer[a])
            done_batch.append(self.terminal_buffer[a])
        return state_batch, action_batch, reward_batch, new_state_batch, done_batch





Default DQN arquitecture

In [5]:
import math

def size_after_maxpool(img_size,kernel_size, stride):
    ret = math.floor(1+(img_size-kernel_size)/stride)
    return ret

class DQN(nn.Module):

    def __init__(self, inputs, outputs):
        super(DQN, self).__init__()
        
        self.input_size=inputs;
        self.output_size=outputs;
        self.imgSize=96;
        
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=self.input_size, 
                      out_channels=64, 
                      kernel_size=5,
                      stride=1,
                      padding=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=64,
                      out_channels=8, 
                      kernel_size=5,
                      stride=1,
                      padding=2),
            nn.ReLU(),
        )
        
        self.layers = nn.Sequential(
            nn.Linear(in_features=self.imgSize*self.imgSize*8, out_features=64),            #Add code here for DQN 
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=self.output_size),            #Add code here for DQN 
        )
        
       
        self.layersdueling = nn.Sequential(
            #Add code here for Dueling 
        )


        self.advantage = nn.Sequential(
            #Add code here for Dueling 
        )
        
        self.value = nn.Sequential(
           #Add code here for Dueling 
        )
  

###### Dueling configuration 
                                                    #############
###########    ############    #################    # advantage #           
#         #    #          #    #               # -> #############    
#   IMG   # -> # features # -> # layersdueling # -> #############
#         #    #  (CONVs) #    #     (FCs)     #    #   value   #
###########    ############    #################    #############
 

    def forward(self, x):
    
        if(useDueling):
            x=self.features(x)
            x = x.view(x.size(0), -1)
            x = self.layersdueling(x)
            advantage = self.advantage(x)
            value     = self.value(x)
            return value + advantage  - advantage.mean()
        else:
            x=self.features(x)
            x = x.view(x.size(0), -1)
            return self.layers(x)

    def policy(self,state):
       with torch.no_grad():
            return self.__call__(state.unsqueeze(0)).argmax()
     
    def getPolicy(self,state,eps_threshold):
        sample = random.random()
        if sample > eps_threshold:
            with torch.no_grad():
                return self.__call__(state.unsqueeze(0)).argmax()
        else:
            return  torch.tensor(random.randrange(self.output_size), device=device, dtype=torch.long)


DQN movie generation (for visual evaluation in Google Colab)

In [6]:
def createMovie(Network,path,Filename,UseClone):
  
    env1 = gym.make("CarRacing-v0")
    envX = wrappers.Monitor(env1,path+'/'+Filename,force=True)
    
    if(UseClone):
       envX.seed((STUDENTID1+STUDENTID2)%100);
    else:
       envX.seed(time.time_ns());
    new_state = envX.reset()


    
    stackedStateX=[np.rollaxis(new_state, 2, 0).copy() for i in range(FrameStack)];
    state =torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameSize*FrameStack,96,96)) ).float().to(device)


    i=0
    Network.eval()
    resetStatus=0;   

    while True:
        envX.render()
        
        action = Network.policy(state);
        reward=0
        for x in range(ControlSteps):
            new_state, r, done, _ = envX.step(action_space[action.item()])
            reward+=r
            if(done):
                break
        if reward<0 and i>InitCounter:
            resetStatus=resetStatus+1;
        else:
            resetStatus=0
        
        stackedStateX.pop(0)
        
 
        stackedStateX.append(np.rollaxis(new_state, 2, 0).copy())
        state=torch.from_numpy(np.reshape(np.array(stackedStateX),(FrameSize*FrameStack,96,96))).float().to(device)
        i=i+1;     
        if done or resetStatus>=ResetCounter: 
          break;

    envX.close()
    env1.close();
    Network.train()
    mp4list = glob.glob(path+'/'+Filename+'/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        if (Running_in_colab):
            ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{0}" type="video/mp4" />
                </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")

Dummy Net with random policy

In [7]:
class RandomNet(nn.Module):

  def __init__(self,outputs):
      super(RandomNet, self).__init__()        
      self.output_size=outputs;

  def forward(self, x):
      return  x

  def policy(self,state):
      return  torch.tensor([[random.randrange(self.output_size)]], device=device, dtype=torch.long)

Parameters

In [8]:
action_space    = [
            (-1  , 1  , 0.2), (0, 1, 0.2), (1   , 1  , 0.2), #           Action Space Structure
            (-1  , 1  ,   0), (0, 1,   0), (1   , 1  ,   0), #        (Steering Wheel, Gas, Break)
            (-1  , 0  , 0.2), (0, 0, 0.2), (1   , 0  , 0.2), # Range        -1~1       0~1   0~1
            (-1  , 0  ,   0), (0, 0,   0), (1   , 0  ,   0)];
            # (-0.5, 0  ,   0),              (0.5 , 0  ,   0),
            # (-0.5, 0.5,   0),              (0.5 , 0.5,   0),
            # (-0.5, 0  , 0.5),              (0.5 , 0  , 0.5)];

 
n_actions=len(action_space)

#hyper-parameters
TotalEpisodes=1001;
MaxSteps=2000;
ControlSteps=3; # number of iterations the same action is executed in the environment 
FrameStack=3 # number of consecutive frames used to represent the state
FreezeCounter=25; # clone the model every X episodes
BatchSize=64;
exploration_threshold=1
exploration_threshold_min=0.01
exploration_decay=0.997
discount_factor=0.95
LearningRate=0.0001
LearningRateDecay=1
 
SaveAtCounter=25 # save model or video at every X episodes
ResetCounter=50 # maximum number of experiences with bad performance
InitCounter=20 # minimum number of experiences per episode
FrameSize=3; # 3 if RGB 1 if grayscale


# use these flags to enable Double DQN and Duelinng (Dueling DQN or Double Dueling DQN)
usedoubleDQN=False
useDueling=False

UseClonedTrack=False # forces the same Track for all episodes

Run the new netwwork with random policy

In [9]:
# randomnet=RandomNet(n_actions)
# createMovie(randomnet,"random",False)

Initialization and Parameters:


In [10]:
env = gym.make("CarRacing-v0")
spec = gym.spec("CarRacing-v0")


#network DQN
policy_net = DQN(FrameSize*FrameStack, n_actions).to(device)
target_net = DQN(FrameSize*FrameStack, n_actions).to(device)

policy_net.load_state_dict(torch.load("bestmodel.ckpt"))

target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

buffer = ReplayBuffer(5000);  # max number of experiences in the buffer (may need to be smaller if CUDA memory errors occur).

optimizer = torch.optim.Adam(policy_net.parameters(), lr=LearningRate)
scheduler = optim.lr_scheduler.ExponentialLR(optimizer,gamma=LearningRateDecay)

loss=  torch.nn.SmoothL1Loss()


DQN training

In [11]:
def trainModel(buffer_data):
    if buffer_data.counter < BatchSize:
        return 0.0

    state_batch, action_batch, reward_batch, new_state_batch, done_batch = buffer_data.sample_buffer(BatchSize)
    state_batch = torch.stack(state_batch).to(device)
    action_batch = torch.stack(action_batch).reshape(-1,1).to(device)
    new_state_batch = torch.stack(new_state_batch).to(device)
    reward_batch = torch.stack(reward_batch).reshape(-1,1).to(device)
    done_batch = torch.stack(done_batch).reshape(-1,1).to(device)

 
    if usedoubleDQN:
        #Add Double DQN code here
        print('Double DQN was implemented for the first part!')

    else: 
        q_actual = torch.gather(policy_net(state_batch),1,index=action_batch)
        with torch.no_grad():
            q_max_next, _ = target_net(new_state_batch).max(dim=1)
            q_max_next=q_max_next.reshape(-1, 1)

        q_target = (q_max_next * discount_factor)*(1-done_batch) + reward_batch
        
        ll=loss(q_actual, q_target)

    optimizer.zero_grad()
    ll.backward()
    torch.nn.utils.clip_grad_norm_(policy_net.parameters(), 1)
    optimizer.step()

    return ll.item();

In [ ]:
loss_val,scores, episodes,events, avg_scores,avg_scores20,exploration = [],[],[], [],[], [], []

bestScore=-99999;
at=0;
bestNet=copy.deepcopy(policy_net);

file_name = mt.set_name("arch", [], BatchSize, exploration_threshold, exploration_threshold_min, exploration_decay, discount_factor, LearningRate, LearningRateDecay)
file_path_ = mt.create_dir("results",file_name)
for f in range(TotalEpisodes):
    print("Episode: "+str(f)+" of "+str(TotalEpisodes))
    done  = False
    score = 0.0
    tloss = 0.0

    # disable to train using random tracks
    if(UseClonedTrack):
       env.seed((STUDENTID1+STUDENTID2)%100);
    else:
       env.seed(time.time_ns());
    
    new_state=env.reset()
   
 
    stackedState=[np.rollaxis(new_state, 2, 0).copy() for i in range(FrameStack)];
    state =torch.from_numpy(np.reshape(np.array(stackedState),(FrameSize*FrameStack,96,96)) ).float().to(device)
   
    if f % FreezeCounter == 0:
       print("########################################"+str(f)+" of "+str(TotalEpisodes))
       target_net.load_state_dict(policy_net.state_dict())
       
    if f % SaveAtCounter == 0:
      #torch.save(policy_net.state_dict(), "v2CarRacing_"+str(f)+'_model.ckpt')
       createMovie(policy_net,file_path_,"CarRacing_"+str(f),UseClonedTrack)
       
    resetStatus=0;
    i=0    
    
    for F in range(MaxSteps):
        #env.render()
        action = policy_net.getPolicy(state,exploration_threshold)
        reward=0
        for _ in range(ControlSteps):
            new_state, r, done, info = env.step(action_space[action.item()])
            reward+=r
            i=i+1
            if(done):
                break
        if reward<0 and i>InitCounter:
            resetStatus=resetStatus+1;
        else:
            resetStatus=0
            
        stackedState.pop(0)
        stackedState.append(np.rollaxis(new_state, 2, 0).copy())
        new_state=torch.from_numpy(np.reshape(np.array(stackedState),(FrameSize*FrameStack,96,96))).float().to(device)
        
        score += reward
        if(F<(MaxSteps-1)):
            buffer.store_tuples(state, action, torch.tensor(reward), new_state, torch.tensor(int(done)))
            
        state = new_state
        trainModel(buffer)


        if(done or resetStatus>=ResetCounter):
            break      

        if(score < -5): break;  
    exploration_threshold= exploration_threshold*exploration_decay if exploration_threshold > exploration_threshold_min else exploration_threshold_min

    if(score>bestScore):
        print(score,F)
        bestScore=score;
        bestNet=copy.deepcopy(policy_net);
        at=f;
    
    exploration.append(exploration_threshold)
    scores.append(score)
    episodes.append(f)
    events.append(F)
    avg_scores.append(score/F)
    avg_scores20.append(np.mean(scores[-20:]))

    if avg_scores20[-1] > 250: break;


torch.save(bestNet.state_dict(), "BestCarRacing_"+str(f)+'_'+str(bestScore)+'_model.ckpt')


Episode: 0 of 1001
Track generation: 1115..1407 -> 292-tiles track
retry to generate track (normal if there are not manyinstances of this message)
Track generation: 1010..1274 -> 264-tiles track
########################################0 of 1001
Track generation: 1148..1439 -> 291-tiles track


14.629657794676572 115
Episode: 1 of 1001
Track generation: 1013..1275 -> 262-tiles track


In [ ]:
import importlib
importlib.reload(eval)

# from PartI_lib import performance_evaluation as eval
eval.performance_evaluation(file_path_,episodes, scores, events, avg_scores, avg_scores20, exploration,[])
eval.report(file_path_,"arch",BatchSize,exploration_threshold,exploration_threshold_min,exploration_decay,discount_factor,
                    LearningRate,LearningRateDecay,episodes, scores, events, avg_scores, avg_scores20, exploration, 0,[])

Plot performance

In [ ]:
figure(figsize=(12, 6), dpi=80)
plt.plot(episodes, scores)
plt.plot(episodes, events)
plt.plot(episodes, avg_scores)
plt.plot(episodes, avg_scores20)
plt.plot(episodes, exploration)
plt.xlabel('episodes')
plt.ylabel('y axis label')
 
plt.title('Track Locked DQN '+str(usedoubleDQN)+' '+str(useDueling))
plt.legend(['scores', 'events', 'avg_scores', 'avg_scores20','exploration'])
plt.show()

View Best Results

In [ ]:
createMovie(bestNet,"results",'bestNet',True)#UseClonedTrack)


